<a href="https://colab.research.google.com/github/eunguru/LinearAlgebra/blob/master/LinearAlgebra_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 선형수학 1주차 - 벡터, 행렬, 텐서, 행렬곱, 역행렬

### 0. 기본 라이브러리, 함수 import

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import numpy as np
import tensorflow as tf


def vector_plot(vecs, xlim, ylim, cols=["#1190FF", "#FF9A13"], alpha=1):
  plt.rc_context({'axes.edgecolor':'orange', 'xtick.color':'red', 'ytick.color':'red'})
  plt.axvline(x=0, color='k', zorder=0)
  plt.axhline(y=0, color='k', zorder=0)
  
  for i in range(len(vecs)):
    if (isinstance(alpha, list)):
      alpha_i = alpha[i]
    else:
      alpha_i = alpha
    x = np.concatenate([[0,0],vecs[i]])
    plt.quiver([x[0]],
               [x[1]],
               [x[2]],
               [x[3]],
               angles='xy', scale_units='xy', scale=1, color=cols[i],
               alpha=alpha_i)
    plt.ylim(-xlim, xlim)
    plt.xlim(-ylim, ylim)
    plt.grid()

def plot_vector2d(vector2d, origin=[0, 0], **options):
  return plt.arrow(origin[0], origin[1], vector2d[0], vector2d[1],
                   head_width=0.2, head_length=0.3, length_includes_head=True,
                   **options)

def plot_transform(P_before, P_after, text_before, text_after, name, color=['#FF9A13', '#1190FF'], axis = [0, 5, 0, 4], arrows=False):
  if arrows:
    for vector_before, vector_after in zip(tf.transpose(P_before), tf.transpose(P_after)):
      plot_vector2d(vector_before, color="#FF9A13", linestyle="--")
      plot_vector2d(vector_after, color="#1190FF", linestyle="-")
      
    plt.rc_context({'axes.edgecolor':'orange', 'xtick.color':'red', 'ytick.color':'red'})
    plt.gca().add_artist(Polygon(tf.transpose(P_before), alpha=0.2))
    plt.gca().add_artist(Polygon(tf.transpose(P_after), alpha=0.3, color="#FF9A13"))
    plt.text(-.25, 1, text_before, size=18, color=color[1])
    plt.text(1.5, 0, text_after, size=18, color=color[0])
    plt.title(name, color='w')
    plt.axis(axis)
    plt.grid()

def evaluate(tensors):
    """Evaluates Tensor or EagerTensor to Numpy `ndarray`s.
    Args:
    tensors: Object of `Tensor` or EagerTensor`s; can be `list`, `tuple`,
      `namedtuple` or combinations thereof.

    Returns:
      ndarrays: Object with same structure as `tensors` except with `Tensor` or
        `EagerTensor`s replaced by Numpy `ndarray`s.
    """
    return tf.nest.pack_sequence_as(tensors,[t.numpy() if tf.is_tensor(t) else t for t in tf.nest.flatten(tensors)])

선형대수는 선형방정식과 선형함수 및 행렬과 벡터 공간을 통한 표현에 관한 수학의 학문분야

머신러닝은 선형대수에 크게 의존, 벡터, 행렬을 이해하는게 중요

In [0]:
import tensorflow as tf
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### 1-1. Scalars, Vectors, Matrices and Tensor
- **Saclars**: 단일숫자
- **Vectors**: 숫자들의 배열
  - 숫자들은 순서대로 정렬되며, 각 숫자들은 순서대로 인덱스에 의해서 식별 가능
  - 벡터는 크기와 방향을 모두 가지는 양을 표현할 수 있는 화살표(화살표의 길의는 크기, 방향은 방향)
- **Matrices**: 행렬은 숫자의 2차원 배열, 각 요소는 하나가 아닌 두개의 인덱스로 식별
  - 행렬 A의 높이가 m이고 너비가 n이면,  $A\ in\ \mathbb {R}^{m \times n}$
  - 행렬의 요소를 $A_{m,n}$ 로 식별: $m$은 행, $n$은 열

![Scalars, Vectors, Matrices and Tensors](https://raw.githubusercontent.com/adhiraiyan/DeepLearningWithTF2.0/master/notebooks/figures/fig0201a.png)

- **Tensor**: 가변 축 수를 가진 정규그리드에 배열된 숫자 배열
  - $A_{i,j,k}$: 좌표 $(i, j, k)$의 텐서 $A$
  - 벡터는 세가지 구성요소로 표현 가능: $(x, y, z)$

![Tensors](https://raw.githubusercontent.com/adhiraiyan/DeepLearningWithTF2.0/master/notebooks/figures/fig0201b.PNG)





$$\color{Orange}{C=A+B\ where\ C_{i, j} = A_{i,j} + B_{i,j} \tag{1}}$$
- Tensorflow에서 Tensor
  - Rank 0 Tensor: Scalar
  - Rank 1 Tensor: Vector
  - Rank 2 Tensor: Matrix
  - Rank 3 Tensor: 3-Tensor
  - Rank n Tensor: n-Tensor

In [0]:
# ones 3x3 rank 2 tensor
rank_2_tensor_A = tf.ones([3,3], name='MatrixA')
print("3x3 Rank 2 Tensor A: \n{} \n".format(rank_2_tensor_A))

# 3x3 rank 2 tensor
rank_2_tensor_B = tf.constant([[1,2,3], [4,5,6], [7,8,9]], name='MatrixB', dtype=tf.float32)
print("3x3 Rank 2 Tensor B: \n{} \n".format(rank_2_tensor_B))

# add two tensor
rank_2_tensor_C = tf.add(rank_2_tensor_A, rank_2_tensor_B, name='MatrixC')
print("Rank 2 Tensor C with shape={} and elements: \n{}".format(rank_2_tensor_C.shape, rank_2_tensor_C))

3x3 Rank 2 Tensor A: 
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]] 

3x3 Rank 2 Tensor B: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]] 

Rank 2 Tensor C with shape=(3, 3) and elements: 
[[ 2.  3.  4.]
 [ 5.  6.  7.]
 [ 8.  9. 10.]]


In [0]:
two_by_three = tf.ones([2,3])
print("2x3 Rank 2 Tensor two_by_three: \n{} \n".format(two_by_three))

try:
  incompatible_tensor = tf.add(two_by_three, rank_2_tensor_B)
except:
  print("Imcompatible shapes to add with two_by_three of shape {}\
  and 3x3 Rank 2 Tensor B of shape{}".format(two_by_three.shape, rank_2_tensor_B.shape))

2x3 Rank 2 Tensor two_by_three: 
[[1. 1. 1.]
 [1. 1. 1.]] 

Imcompatible shapes to add with two_by_three of shape (2, 3)  and 3x3 Rank 2 Tensor B of shape(3, 3)


행렬의 각 요소에 대해 해당 작업을 수행화여 핼렬에 스칼라를 추가하거나 행렬에 스칼라를 곱할 수도 있음
$$\color{orange}{D=a \cdot B+c\ where\ D_{i,j}=a \cdot B_{i,j}+c \tag{2}}$$

In [0]:
# scalar a,c and Matrix B
rank_0_tensor_a = tf.constant(2, name="scalar_a", dtype=tf.float32)
rank_2_tensor_B = tf.constant([[1,2,3], [4,5,6], [7,8,9]], name="MatrixB", dtype=tf.float32)
rank_0_tensor_c = tf.constant(3, name="scalar_c", dtype=tf.float32)

# aB
multiply_scalar = tf.multiply(rank_0_tensor_a, rank_2_tensor_B)

# aB+c
rank_2_tensor_D = tf.add(multiply_scalar, rank_0_tensor_c, name="MatrixD")

print("""Original Rank 2 Tensor B: \n{0} \n\nScalar a: {1} \n\
Rank 2 Tensor for aB: \n{2} \n\nScalar c: {3} \n\
Rank 2 Tensor D = aB+c: \n{4} )""".format(rank_2_tensor_B, rank_0_tensor_a, 
                                          multiply_scalar, rank_0_tensor_c, rank_2_tensor_D))

Original Rank 2 Tensor B: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]] 

Scalar a: 2.0 
Rank 2 Tensor for aB: 
[[ 2.  4.  6.]
 [ 8. 10. 12.]
 [14. 16. 18.]] 

Scalar c: 3.0 
Rank 2 Tensor D = aB+c: 
[[ 5.  7.  9.]
 [11. 13. 15.]
 [17. 19. 21.]] )


행렬에서 중요한 것: **Tanspose(전치)**

행렬의 Transpose(전치)는 **main diagonal**이라고 하는 대각선을 따라 행과 열을 바꾼 행렬

$A$행렬을 $A\ top$으로 바꾸고 아래와 같이 정의 함
$$(A^\top)_{i,j} = A_{j,i}$$

In [0]:
# rank 2 Matrix E
rank_2_tensor_E = tf.constant([[1,2,3], [4,5,6]])

# transpose Matrix E
transpose_E = tf.transpose(rank_2_tensor_E, name="transposeE")

print("""Rank 2 Tensor E of shape: {0} and elements: \n{1}\n\
Tanspose of Rank 2 Tensor E of shape: {2} and elements: \n{3}""".format(rank_2_tensor_E.shape, rank_2_tensor_E,
                                                                        transpose_E.shape, transpose_E))

Rank 2 Tensor E of shape: (2, 3) and elements: 
[[1 2 3]
 [4 5 6]]
Tanspose of Rank 2 Tensor E of shape: (3, 2) and elements: 
[[1 4]
 [2 5]
 [3 6]]


딥러닝에서는 행렬과 벡터를 추가하여 $C_{i,j} = A_{i,j}+b_j$인 다른 행렬을 생성 할 수 있음

즉, 벡터 b가 행렬의 각 행에 추가되며, 여러위치에 b를 암시적으로 복사하는 것을 **broadcasting**이라고 함

In [0]:
# rank 1 vector b
rank_1_tensor_b = tf.constant([[4.], [5.], [6.]])
# broadcast, F = A + b
rank_2_tensor_F = tf.add(rank_2_tensor_A, rank_1_tensor_b, name="broadcastF")

print("""Rank 2 tensor A: \n{0}\n \nRank 1 Tensor b: \n{1}\
\nRank 2 tensor F = A+b: \n{2}""".format(rank_2_tensor_A, rank_1_tensor_b, rank_2_tensor_F))

Rank 2 tensor A: 
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]
 
Rank 1 Tensor b: 
[[4.]
 [5.]
 [6.]]
Rank 2 tensor F = A+b: 
[[5. 5. 5.]
 [6. 6. 6.]
 [7. 7. 7.]]


### 1-2. Multiplying Matrices and Vectors
행렬 $A$와 $B$, $A$의 행렬곱을 정의하려면 $A$에 $B$와 같은 수의 열이 있어야 함

$A$의 모양이 $m\times n$이고 $B$의 모양이 $n \times p$이면 $C$의 모양은 $m \times p$
$$\color{orange}{C_{i,j}=\sum_kA_{i,k}B_{k,j} \tag{3}}$$

In [0]:
# Matrix A shape: (2,3) B shape: (3,4)
mmv_matrix_A = tf.ones([2,3], name="matrix_A")
mmv_matrix_B = tf.constant([[1,2,3,4], [1,2,3,4], [1,2,3,4]], name="matrix_B", dtype=tf.float32)

# Matrix C: C=AB, C shape: (2,4)
matrix_multiply_C = tf.matmul(mmv_matrix_A, mmv_matrix_B, name="matrix_multiply_C")

print("""Matrix A: shape {0} \nelements: \n{1}\
\n\nMatrix B: shape {2} \nelements: \n{3}\
\nMatrix C: shape {4} \nelements: \n{5}""".format(mmv_matrix_A.shape, mmv_matrix_A,
                                                  mmv_matrix_B.shape, mmv_matrix_B,
                                                  matrix_multiply_C.shape, matrix_multiply_C))

Matrix A: shape (2, 3) 
elements: 
[[1. 1. 1.]
 [1. 1. 1.]]

Matrix B: shape (3, 4) 
elements: 
[[1. 2. 3. 4.]
 [1. 2. 3. 4.]
 [1. 2. 3. 4.]]
Matrix C: shape (2, 4) 
elements: 
[[ 3.  6.  9. 12.]
 [ 3.  6.  9. 12.]]


개별 요소의 곱을 포함하는 행렬을 얻으려면 **element wise production** 또는 **Handmamard product**를 사용하고 $A \odot B$으로 표기

In [0]:
# Matrix A, B shape (3,3)
element_matrix_A = tf.ones([3,3], name="element_matrix_A")
element_matrix_B = tf.constant([[1,2,3], [4,5,6], [7,8,9]], name="element_matrix_B", dtype=tf.float32)

# element wise product
element_wise_C = tf.multiply(element_matrix_A, element_matrix_B, name="element_wise_C")

print("""Matrix A: shape {0} \nelements: \n{1}\
\n\nMatrix A: shape {2} \nelements: \n{3}\
\nMatrix C: shape {4} \nelements: \n{5}""".format(element_matrix_A.shape, element_matrix_A,
                                                  element_matrix_B.shape, element_matrix_B,
                                                  element_wise_C.shape, element_wise_C))

Matrix A: shape (3, 3) 
elements: 
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

Matrix A: shape (3, 3) 
elements: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
Matrix C: shape (3, 3) 
elements: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]


$A$와 $B$사이의 내적을 계산하기 위해 $A$의 행 $i$와 $B$의 $j$사이의 내적으로 $C_{i,j}를 계산$

![Dot Product](https://raw.githubusercontent.com/adhiraiyan/DeepLearningWithTF2.0/master/notebooks/figures/fig0202b.jpg)

In [0]:
# Matrix A, B shape: (3,3)
dot_matrix_A = tf.ones([3,3], name="dot_matrix_A")
dot_matrix_B = tf.constant([[1,2,3], [4,5,6], [7,8,9]], name="dot_matrix_B", dtype=tf.float32)

# Dot Product AB
dot_product_C = tf.tensordot(dot_matrix_A, dot_matrix_B, axes=1, name="dot_product_C")

print("""Matrix A: shape {0} \nelements: \n{1}\
\n\nMatrix B: shape {2} \nelements: \n{3}\n\
Matrix C: shape {4} \nelements: \n{5}""".format(dot_matrix_A.shape, dot_matrix_A,
                                                dot_matrix_B.shape, dot_matrix_B,
                                                dot_product_C.shape, dot_product_C))

Matrix A: shape (3, 3) 
elements: 
[[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]

Matrix B: shape (3, 3) 
elements: 
[[1. 2. 3.]
 [4. 5. 6.]
 [7. 8. 9.]]
Matrix C: shape (3, 3) 
elements: 
[[12. 15. 18.]
 [12. 15. 18.]
 [12. 15. 18.]]


행렬 곱셈의 일부 속성(분산속성): $$\color{orange}{A(B+C)=AB+AC\tag{4}}$$

In [0]:
matrix_A = tf.constant([[1,2], [3,4]], name="matrix_a")
matrix_B = tf.constant([[5,6], [7,8]], name="matrix_b")
matrix_C = tf.constant([[9,1], [2,3]], name="matrix_c")

In [0]:
print("Matrix A: \n{} \n\nMatrix B: \n{} \n\nMatrix C: \n{}\n".format(matrix_A, matrix_B, matrix_C))

# AB+AC
distributive_RHS = tf.add(tf.matmul(matrix_A, matrix_B), tf.matmul(matrix_A, matrix_C), name="RHS")

# A(B+C)
distributive_LHS = tf.matmul(matrix_A, (tf.add(matrix_B, matrix_C)), name="LHS")

predictor = tf.reduce_all(tf.equal(distributive_RHS, distributive_LHS))

def true_print():
  print("""Distributive property is valid RHS: AB+AC: \n{} \n\nLHS: A(B+C): \n{}""".format(distributive_RHS, distributive_LHS))

def false_print():
  print("""You Broke the Distributive Property of Matrix RHS: AB+AC: \n{}\
  \n\nis NOT Equal to LHS: A(B+C): \n{}""".format(distributive_RHS, distributive_LHS))

tf.cond(predictor, true_print, false_print)

Matrix A: 
[[1 2]
 [3 4]] 

Matrix B: 
[[5 6]
 [7 8]] 

Matrix C: 
[[9 1]
 [2 3]]

Distributive property is valid RHS: AB+AC: 
[[32 29]
 [78 65]] 

LHS: A(B+C): 
[[32 29]
 [78 65]]


행렬 곱셈의 일부 속성(연관속성): $$\color{orange}{A(BC)=(AB)C\tag{5}}$$

In [0]:
print("Matrix A: \n{} \n\nMatrix B: \n{} \n\nMatrix C: \n{}\n".format(matrix_A, matrix_B, matrix_C))

# (AB)C
assosiative_RHS = tf.matmul(tf.matmul(matrix_A, matrix_B), matrix_C)
# A(BC)
assosiative_LHS = tf.matmul(matrix_A, tf.matmul(matrix_B, matrix_C))

predictor = tf.reduce_all(tf.equal(assosiative_RHS, assosiative_LHS))

def true_print():
  print("""Assosiative property is valid RHS: (AB)C: \n{} \n\nLHS: A(BC): \n{}""".format(assosiative_RHS, assosiative_LHS))

def false_print():
  print("""You Broke the Assosiative Property of Matrix RHS: (AB)C): \n{}\
  \n\nis NOT Equal to LHS: A(BC): \n{}""".format(assosiative_RHS, assosiative_LHS))

tf.cond(predictor, true_print, false_print)


Matrix A: 
[[1 2]
 [3 4]] 

Matrix B: 
[[5 6]
 [7 8]] 

Matrix C: 
[[9 1]
 [2 3]]

Assosiative property is valid RHS: (AB)C: 
[[215  85]
 [487 193]] 

LHS: A(BC): 
[[215  85]
 [487 193]]


행렬 곱셈의 일부 속성 $$\color{orange}{AB \neq BA\tag{6}}$$

In [0]:
print("Matrix A: \n{} \n\nMatrix B: \n{}".format(matrix_A, matrix_B))

# Matrix A times B
commutative_RHS = tf.matmul(matrix_A, matrix_B)

# Matrix B times A
commutative_LHS = tf.matmul(matrix_B, matrix_A)

predictor = tf.logical_not(tf.reduce_all(tf.equal(commutative_RHS, commutative_LHS)))

def true_print():
  print("""Matrix Multiplication is not commutative  (AB): \n{} \n\nLHS: (BA): \n{}""".format(commutative_RHS, commutative_LHS))

def false_print():
  print("""You made Matrix Multipliccation commutative RHS: (AB): \n{}\
  \n\n LHS: (BA): \n{}""".format(commutative_RHS, commutative_LHS))

tf.cond(predictor, true_print, false_print)


Matrix A: 
[[1 2]
 [3 4]] 

Matrix B: 
[[5 6]
 [7 8]]
Matrix Multiplication is not commutative  (AB): 
[[19 22]
 [43 50]] 

LHS: (BA): 
[[23 34]
 [31 46]]


행렬 곱셈의 일부 속성 (Transpose): $$\color{orange}{(AB)^\top = B^\top A^\top\tag{7}}$$

In [0]:
print("Matrix A: \n{} \n\nMatrix B: \n{}\n".format(matrix_A, matrix_B))

transpose_RHS = tf.transpose(tf.matmul(matrix_A, matrix_B))

transpose_LHS = tf.matmul(matrix_B, matrix_A, transpose_a=True, transpose_b=True)

predictor = tf.reduce_all(tf.equal(transpose_RHS, transpose_LHS))

def true_print():
  print("""Transpose property is valid RHS: (AB)^T: \n{}\
  \n\nLHS: (B^T A^T): \n{}""".format(transpose_RHS, transpose_LHS))

def false_print():
  print("""You Broken the Transpose property of Matrix RHS: (AB)^T: \n{}\
  \n\nLHS: (B^TA^T): \n{}""".format(transpose_RHS, transpose_LHS))

tf.cond(predictor, true_print, false_print)

Matrix A: 
[[1 2]
 [3 4]] 

Matrix B: 
[[5 6]
 [7 8]]

Transpose property is valid RHS: (AB)^T: 
[[19 43]
 [22 50]]  

LHS: (B^T A^T): 
[[19 43]
 [22 50]]


### 1-3. Identity and Inverse Matrices
선형대수학은 **matrix inversion**이라는 강력한 도구를 제공

$A$의 많은 값에 대해 $Ax=b$를 분석적으로 해결 가능

행렬 반전 설명을 위해서 먼저 **identity matrix**개념을 정의해야 함

항등행렬은 벡터에 해당 행렬을 곱할 때 벡터를 변경하지 않는 행렬
$$\color{orange}{I_n \in  \mathbb{R}^{n \times m} \text{and}\ \forall x \in \mathbb{R}^n, I_nx = x \tag{8}}$$

항등행렬의 구조는 주 대각선을 따른은 모든 항목은 1이고 모든 항목은 0 

In [0]:
# matrix I
identity_matrix_I = tf.eye(3,3, dtype=tf.float32, name='IdentityMatrixI')
print("Identity matrix I: \n{}\n".format(identity_matrix_I))

iim_vector_x = tf.constant([[4], [5], [6]], name='Vector_x', dtype=tf.float32)
print("Vector x: \n{}\n".format(iim_vector_x))

iim_matrix_C = tf.matmul(identity_matrix_I, iim_vector_x, name='MatrixC')
print("Matrix C from Ix: \n{}".format(iim_matrix_C))

Identity matrix I: 
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]

Vector x: 
[[4.]
 [5.]
 [6.]]

Matrix C from Ix: 
[[4.]
 [5.]
 [6.]]


$A$의 **matrix inverse**는 $A^{-1}$로 표시되며 다음과 같이 정의
$$\color{orange}{A^{-1}A = I_n\tag{9}}$$



In [0]:
iim_matrix_A = tf.constant([[2,3], [2,2]], name='MatrixA', dtype=tf.float32)

try:
  # inverse
  inverse_matrix_A = tf.linalg.inv(iim_matrix_A)

  # identity matrix: tf.eye
  identity_matrix = tf.eye(2,2, dtype=tf.float32, name="identity")

  iim_RHS = identity_matrix
  iim_LHS = tf.matmul(inverse_matrix_A, iim_matrix_A, name="LHS")

  predictor = tf.reduce_all(tf.equal(iim_RHS, iim_LHS))

  def true_print():
    print("""A^-1 times A equals the Identity Matrix\
    Matrix A: \n{0} \n\nInverse of Matrix A: \n{1}\
    \n\nRHS: I: \n{2}\
    \n\nLHS: A^(-1) A: \n{3}""".format(iim_matrix_A,
                                       inverse_matrix_A,
                                       iim_RHS,
                                       iim_LHS))
  def false_print():
    print("Condition Failed")
  
  tf.cond(predictor, true_print, false_print)

except:
  print("""A^-1 doesnt exist\
  Matrix A: \n{} \n\nInverse of Matrix A: \n{}\
  \n\nRHS: I: \n{}\
  \nLHS: (A^(-1) A): \n{}""".format(iim_matrix_A, inverse_matrix_A, iim_RHS, iim_LHS))

A^-1 times A equals the Identity Matrix    Matrix A: 
[[2. 3.]
 [2. 2.]] 

Inverse of Matrix A: 
[[-1.   1.5]
 [ 1.  -1. ]]    

RHS: I: 
[[1. 0.]
 [0. 1.]]    

LHS: A^(-1) A: 
[[1. 0.]
 [0. 1.]]


$Ax=b$방정식은 다음과 같이 풀 수 있음
$$\color{orange}{A^{-1}Ax=A^{-1}b \\
I_nx=A^{-1}b \\
x=A^{-1}b\tag{10}}$$

$A^{-1}는 다음과 같이 찾을수 있음$

![Matrix Inverse](https://raw.githubusercontent.com/adhiraiyan/DeepLearningWithTF2.0/master/notebooks/figures/fig0203a.PNG)


In [0]:
sys_matrix_A = tf.constant([[2,3], [4,9]], dtype=tf.float32)
sys_vector_B = tf.constant([[6], [15]], dtype=tf.float32)

print("Matrix A: \n{} \n\nVector B: \n{}\n".format(sys_matrix_A, sys_vector_B))

# x = A^(-1)b
sys_x = tf.matmul(tf.linalg.inv(sys_matrix_A), sys_vector_B)
print("Vector x is: \n{} \nWhere x = {} and y = {}".format(sys_x, sys_x[0], sys_x[1]))

Matrix A: 
[[2. 3.]
 [4. 9.]] 

Vector B: 
[[ 6.]
 [15.]]

Vector x is: 
[[1.5      ]
 [1.0000001]] 
Where x = [1.5] and y = [1.0000001]
